In [173]:
import quimb.tensor as qtn
import numpy as np
from local_discriminant import disc_parts
import pickle
from options import options
import os


define S-transform

get disc parts

Generate gibbs state (imag time evo/ dmrg)

Define linear operator from fmrg like contaction (check for center of orthogonalities, to avoid full contraction)

project out the gibbs state

use iterative eigen solver


Tasks

Easy backand switching (cpu, gpu) or (torch, cuda, numpy)

Find out how quimb linear operators work

COMMENT EVERYTHING

In [174]:
def sh_weight(x,a=39):
    if x==0:
        x=10e-9
    return 1.j*(1-np.cos(a*x))/np.sinh(2*np.pi*x)


def ch_weight(x):
    return 1./np.cosh(2*np.pi*x)

In [175]:
def kraus_mps(jump_op,site_number,length):

    mps_sites=[np.eye(2).reshape(4)]*length
    mps_sites[site_number]=jump_op.reshape(4)
    return qtn.MPS_product_state(mps_sites)


def id_mps(length):
    return qtn.MPS_product_state([np.eye(2).reshape(4)]*length)

In [ ]:
from mpo_utils import mirror_mpo
def generate_disc_parts(ham_tp,ham_tm,delta_t,num_steps,jump_ops,folder,options=options):
    length=ham_tp.L
    scas={key:[] for key in jump_ops}
    scds={key:[] for key in jump_ops}
    if not os.path.exists(folder):
        os.makedirs(folder)
    for i in range(int(length/2)):
        for item in jump_ops.items():
            print(f'Site: {item[0]}_{i}')
            #generate parts
            scd_a,sca_a=disc_parts(item[1],i,length,delta_t,num_steps,ham_tp,ham_tm,opts=options,gpu=False)
            scas[item[0]].append(sca_a)
            scds[item[0]].append(scd_a)
            #save
            with open(folder+f'scas.pkl','wb') as f:
                pickle.dump(scas,f)
            with open(folder+f'scds.pkl','wb') as f:
                pickle.dump(scds,f)
    #Mirror the data to get the full system

    for item in scas.items():
        key=item[0]
        lst=item[1]
        scas[key]=lst+ [mirror_mpo(sca) for sca in reversed(lst)]
    for item in scds.items():
        key=item[0]
        lst=item[1]
        scds[key]=lst+ [mirror_mpo(scd) for scd in reversed(lst)]
    with open(folder+f'scas.pkl','wb') as f:
        pickle.dump(scas,f)
    with open(folder+f'scds.pkl','wb') as f:
        pickle.dump(scds,f)
    return scds,scas
    

In [177]:
from mpo_utils import mirror_mpo
def load_disc_parts(folder):
    
    with open(folder+'scas.pkl', 'rb') as file:
        scas = pickle.load(file)
    with open(folder+'scds.pkl', 'rb') as file:
        scds = pickle.load(file)
    
    return scds,scas

In [ ]:
L=4 #use even numbers
J=1
h=1.5
sv_tol=1e-8
delta_t=.08
final_t=3.2

a=33 #0.05 33 #0.1 #32
num_steps=int(final_t/delta_t)


I=np.eye(2)
X=np.array([[0,1],[1,0]])
Y=np.array([[0,-1.j],[1.j,0]])
Z=np.array([[1,0],[0,-1]])

h2=-J*(np.kron(np.kron(Z,I),np.kron(Z,I))-np.kron(np.kron(I,Z),np.kron(I,Z)))
h1=h*(np.kron(X,I)-np.kron(I,X))

#ham_tp=qtn.LocalHam1D(L,H2=h2,H1=h1,cyclic=False)
#ham_tm=qtn.LocalHam1D(L,H2=-h2,H1=-h1,cyclic=False)

weight_func=lambda x: sh_weight(x,a)
tol=sv_tol
options={}
options['split_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel','max_bond':100}
options['tebd_opts']={'tol':tol,'order':4,'progbar':False}
options['comp_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel'}

paulis={'x':X,'z':Z}

In [179]:
ham_tp=qtn.LocalHam1D(L,H2=h2,H1=h1,cyclic=False)
ham_tm=qtn.LocalHam1D(L,H2=-h2,H1=-h1,cyclic=False)



In [180]:

generate_disc_parts(ham_tp,ham_tm,delta_t,num_steps,jump_ops=paulis,name=f'tfim_J{J}_h{h}')

Site: x_0
sa 0 done
scd_p1 0 done
scasa 0 done
scasca 0 done
Site: z_0
sa 0 done
scd_p1 0 done
scasa 0 done
scasca 0 done
Site: x_1
sa 1 done
scd_p1 1 done
scasa 1 done
scasca 1 done
Site: z_1
sa 1 done
scd_p1 1 done
scasa 1 done
scasca 1 done


({'x': [MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=15),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=15)],
  'z': [MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6)]},
 {'x': [MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6)],
  'z': [MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=2),
   MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=10),
   MatrixProductOperator(tensors=4, indices=1

In [ ]:
folder=f'../DATA/tfim_J{J}_h{h}_L{L}_{''.join(jump_ops.keys())}//'
scds,scas=load_disc_parts(folder)

In [182]:
scds

{'x': [MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=15),
  MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
  MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
  MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=15)],
 'z': [MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
  MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
  MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=16),
  MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6)]}

In [183]:
new_scas=[]
for item in scas.items():
    new_scas+=item[1]



In [184]:
new_scas

[MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
 MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
 MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
 MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=6),
 MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=2),
 MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=10),
 MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=10),
 MatrixProductOperator(tensors=4, indices=11, L=4, max_bond=2)]